# Lunar Lander with Cross-Entropy Method

In this notebook we look at the lunar lander environment and solve it with the cross-entropy method.

In [1]:
#!pip3 install 'gymnasium[box2d]'

In [2]:
import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import matplotlib.pyplot as plt
%matplotlib inline
from collections import deque

torch.manual_seed(1)
np.random.seed(1)

# Neural Network

We define a simple neural network that generates the action scores based on a given state.

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(obs_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, n_actions)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Generate Episodes

We generate a batch of episodes and remember the traversed states, actions and rewards. To select the next action we use the output of the network. For this we first pass the scores through a softmax to get probabilites. In the second step we sampel from this distribution to get the next action to execute.

In [4]:
def generate_batch(env, batch_size, t_max=5000):
    
    activation = nn.Softmax(dim=1)
    batch_actions,batch_states, batch_rewards = [],[],[]
    
    for b in range(batch_size):
        states, actions = [], []
        total_reward = 0
        s, _ = env.reset(seed=0)
        for t in range(t_max):
            
            s_v = torch.FloatTensor([s])
            act_probs_v = activation(net(s_v))
            act_probs = act_probs_v.data.numpy()[0]
            a = np.random.choice(len(act_probs), p=act_probs)

            new_s, r, done, _, _ = env.step(a)

            # record sessions like you did before
            states.append(s)
            actions.append(a)
            total_reward += r

            s = new_s
            if done:
                batch_actions.append(actions)
                batch_states.append(states)
                batch_rewards.append(total_reward)
                break
                
    return batch_states, batch_actions, batch_rewards

# Training

In the training step, we first use the neural network to generate a batch of episodes and then use the state-action pairs to improve the neural network.

In [5]:
batch_size = 100
session_size = 100
percentile = 80
hidden_size = 200
completion_score = 100
learning_rate = 0.01

env = gym.make("LunarLander-v2")
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(n_states, hidden_size, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=learning_rate)

for i in range(session_size):
    
    # generate new episodes
    states, actions, rewards = generate_batch(env, batch_size, t_max=500)
    
    threshold = np.percentile(rewards, 80)
    
    # train on the states using actions as targets
    for s_i in range(len(states)):
        if rewards[s_i] < threshold: # skip this iteration if rewards are too low
            continue
        optimizer.zero_grad()
        tensor_states = torch.FloatTensor(states[s_i])
        tensor_actions = torch.LongTensor(actions[s_i])
        action_scores_v = net(tensor_states)
        loss_v = objective(action_scores_v, tensor_actions)
        loss_v.backward()
        optimizer.step()

    #show results
    mean_reward = np.mean(rewards)
    print("%d: loss=%.3f, reward_mean=%.1f" % (
            i, loss_v.item(), mean_reward))
    
    #check if 
    if np.mean(rewards)> completion_score:
        print("Environment has been successfullly completed!")

/var/folders/g4/qd5krtcd4bs0p2_c6rcbbb7c0000gn/T/ipykernel_64699/2392018301.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  s_v = torch.FloatTensor([s])


0: loss=1.359, reward_mean=-90.8
1: loss=1.366, reward_mean=-107.0
2: loss=1.372, reward_mean=-115.2
3: loss=1.393, reward_mean=-105.3
4: loss=1.377, reward_mean=-76.5
5: loss=1.379, reward_mean=-62.9
6: loss=1.182, reward_mean=-62.9
7: loss=1.022, reward_mean=-53.6
8: loss=0.995, reward_mean=-50.5
9: loss=1.037, reward_mean=-87.2
10: loss=0.998, reward_mean=-106.4
11: loss=1.074, reward_mean=-95.6
12: loss=1.088, reward_mean=-73.1
13: loss=1.023, reward_mean=-83.6
14: loss=0.960, reward_mean=-54.3
15: loss=0.922, reward_mean=-13.5
16: loss=1.013, reward_mean=10.9
17: loss=0.944, reward_mean=30.3
18: loss=0.982, reward_mean=39.7
19: loss=1.006, reward_mean=50.0
20: loss=0.894, reward_mean=46.2
21: loss=0.907, reward_mean=56.2
22: loss=0.902, reward_mean=49.9
23: loss=0.824, reward_mean=51.0
24: loss=0.865, reward_mean=50.6
25: loss=0.893, reward_mean=50.0
26: loss=0.788, reward_mean=50.1
27: loss=0.878, reward_mean=51.1
28: loss=0.872, reward_mean=59.1
29: loss=0.797, reward_mean=58.1
